In [5]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import zipfile
import os
import requests

In [12]:
# Step 1: Ensure Dataset Directory Exists
train_dir = "C:\\Users\\deepi\\Downloads\\archive (2)\\train"
test_dir = "C:\\Users\\deepi\\Downloads\\archive (2)\\test"

In [13]:
# Step 2: Data Augmentation and Preprocessing
data_gen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [14]:
train_data = data_gen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

test_data = ImageDataGenerator(rescale=1.0/255.0).flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')


Found 11431 images belonging to 4 classes.
Found 7178 images belonging to 7 classes.


In [15]:
# Step 3: Load Pre-trained Model and Fine-Tune
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 9s 0us/step


In [16]:
# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

In [17]:
# Add custom layers
gap = GlobalAveragePooling2D()(base_model.output)
dropout = Dropout(0.5)(gap)
output_layer = Dense(train_data.num_classes, activation='softmax')(dropout)


In [18]:
model = Model(inputs=base_model.input, outputs=output_layer)


In [19]:
# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy'])


In [ ]:
# Step 4: Train the Model
history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=10,
    steps_per_epoch=train_data.samples // train_data.batch_size,
    validation_steps=test_data.samples // test_data.batch_size)

Epoch 1/10


C:\Users\deepi\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 55/357 ━━━━━━━━━━━━━━━━━━━━ 9:06 2s/step - accuracy: 0.2982 - loss: 1.5650

In [ ]:
# Step 5: Fine-Tuning (Unfreezing Layers for Better Performance)
for layer in base_model.layers:
    layer.trainable = True

In [ ]:
# Recompile with a lower learning rate
model.compile(
    optimizer=Adam(learning_rate=0.00001),
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [ ]:
# Continue Training
history_finetune = model.fit(
    train_data,
    validation_data=test_data,
    epochs=10,
    steps_per_epoch=train_data.samples // train_data.batch_size,
    validation_steps=test_data.samples // test_data.batch_size)


In [ ]:
# Step 6: Save the Model
model.save("emotion_recognition_resnet50.h5")

In [ ]:
# Step 7: Evaluate the Model
loss, accuracy = model.evaluate(test_data)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

